In [ ]:
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
import xarray as xr

import eradiate
from eradiate.scenes.surface import mesh_from_dem

eradiate.set_mode("mono")

In [ ]:
# Make data
x = np.arange(-5, 5, 0.25)
y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(x, y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

dem = xr.DataArray(
    data=Z,
    coords={
        "x": ("x", x, {"units": "m"}),
        "y": ("y", y, {"units": "m"}),
    },
    attrs={"units": "m"},
)
dem.plot.imshow()
mesh = mesh_from_dem(dem, geometry="plane_parallel")[0]

In [ ]:
# Make data
x = np.arange(-1, 1, 0.25 / 5)
y = np.arange(-1, 1, 0.25 / 5)
X, Y = np.meshgrid(x * 5, y * 5)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R) / 200

dem = xr.DataArray(
    data=Z,
    coords={
        "lon": ("lon", x, {"units": "deg"}),
        "lat": ("lat", y, {"units": "deg"}),
    },
    attrs={"units": "m"},
)
dem.plot.imshow()
mesh = mesh_from_dem(dem, geometry="plane_parallel", planet_radius=2.0)[0]

In [ ]:
d = 0.05
scene_dict = {
    "type": "scene",
    # "illumination": {"type": "directional", "direction": [0, -1, -1]},
    "illumination": {"type": "constant"},
    "sensor": {
        "type": "perspective",
        "to_world": mi.ScalarTransform4f.look_at(
            origin=[1.5 * d, 1.5 * d, d],
            target=[0, 0, 0],
            up=[0, 0, 1],
            # origin=[0, 0, d], target=[0, 0, 0], up=[0, 1, 0]
        ),
        "far_clip": 1e12,
        "film": {
            "type": "hdrfilm",
            "pixel_format": "luminance",
            "component_format": "float32",
        },
    },
    "integrator": {"type": "path"},
}
scene_dict["mesh"] = mesh.instance
# scene_dict["north"] = {
#     "type": "sphere",
#     "center": mi.Vector3f(0, 100000, 0),
#     "radius": 10000,
# }
# scene_dict["east"] = {
#     "type": "sphere",
#     "center": mi.Vector3f(100000, 0, 0),
#     "radius": 10000,
# }

scene = mi.load_dict(scene_dict)
img = mi.render(scene)

plt.imshow(img)

In [ ]:
mesh.instance